# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = [noise * 2 for noise in NOISE_LIST]

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Loop function

In [4]:
from tqdm import tqdm
import json
from src import cpp_soft_info
import pymatching
import numpy as np

def func(distance, rounds, resets=False, shots = 1e5, p_meas = -1, thresholds = np.linspace(0, 0.5, 11)):
    # Initialize simulator
    simulator = RepCodeIQSimulator(provider, distance, rounds, DEVICE, _is_hex=_is_hex, _resets = resets, other_date=OTHER_DATE)
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST, verbose=False)
    print("Generated IQ data")

    # Run decoding
    result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                        rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                        p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=False, _ntnn_edges = not resets)

    result_predecoders = []
    for threshold in tqdm(thresholds, desc='Decoding for different thresholds'):
        result_predecoders.append(cpp_soft_info.decode_time_nn_predecode_grid(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data, rounds, int(LOGICAL), 
                                                                              resets, simulator.qubit_mapping, simulator.grid_dict, 
                                                                              simulator.processed_scaler_dict, _detailed=False, 
                                                                              threshold=threshold, _ntnn_edges = not resets))
   
    return result_predecoders, result_informed
    # return result_informed


In [5]:
import numpy as np

# distances = np.linspace(5, 31, 10)
distances = [31, 35, 37, 41, 43, 47, 49, 53, 55]
distances

[31, 35, 37, 41, 43, 47, 49, 53, 55]

In [7]:
import numpy as np
from tqdm import tqdm

shots = int(1e6) 
p_meas = -1

file_str = f'../results/threshold_vs_err_rate.json'

for i in tqdm(distances, desc='Simulating for distance'):

    if i < 16:
        continue

    if int(i)%2 == 0: # only simulate odd distances
        i = int(i)+1
    
    i = int(i)

    # Save results
    with open(file_str, 'r') as f:
        data = json.load(f)
   

    distance = i
    rounds = i     


    thresholds = np.linspace(0, 0.5, 31)

    result_predecoders, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas, thresholds=thresholds, resets=_RESETS)
    # result_informed = func(distance, rounds, shots = shots, p_meas=p_meas)

    # num errors predecoder
    num_errors = [result.decode_result.num_errors for result in result_predecoders]
    nb_rm_edges = [np.mean(result.nb_rm_edges) for result in result_predecoders]

    data[str((distance, rounds))] = {'predecoder': {'num_errors': num_errors, 'nb_rm_edges': nb_rm_edges, 
                                                    'thresholds': list(thresholds)}, 'informed': result_informed.num_errors, 
                                                    'shots': shots, 'p_meas': p_meas, 'resets': _RESETS, 'noise': NOISE_LIST}

    # data[str((distance, rounds))] = {'informed': result_informed.num_errors, shots: shots, 'p_meas': p_meas, 'resets': _RESETS, 'noise': NOISE_LIST}

    with open(file_str, 'w') as f:
        json.dump(data, f, indent=4)


Simulating for distance:   0%|          | 0/9 [00:00<?, ?it/s]

Generated IQ data


Simulating for distance:  11%|█         | 1/9 [1:48:26<14:27:32, 6506.51s/it]

Generated IQ data


Simulating for distance:  22%|██▏       | 2/9 [4:12:06<15:04:06, 7749.50s/it]

...

Simulating for distance:  44%|████▍     | 4/9 [10:34:39<14:31:14, 10454.81s/it]

Generated IQ data


Simulating for distance:  56%|█████▌    | 5/9 [15:57:39<12:46:07, 11491.87s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/_3/n7zzdff5239886c9gmhrk6yr0000gn/T/ipykernel_7014/1467620110.py", line 30, in <module>
    result_predecoders, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas, thresholds=thresholds, resets=_RESETS)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/_3/n7zzdff5239886c9gmhrk6yr0000gn/T/ipykernel_7014/2196693173.py", line 10, in func
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST, verbose=False)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/IQ_data